<a href="https://colab.research.google.com/github/sdikici/Demand_Forecasting_Prophet_DeepAR/blob/main/DeepAR_hyperparameter_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sercandikici/merged-dataset-electricty-weather-for-modelling
! unzip merged-dataset-electricty-weather-for-modelling.zip

Dataset URL: https://www.kaggle.com/datasets/sercandikici/merged-dataset-electricty-weather-for-modelling
License(s): unknown
100% 104k/104k [00:00<00:00, 361kB/s]
100% 104k/104k [00:00<00:00, 361kB/s]
Archive:  merged-dataset-electricty-weather-for-modelling.zip
  inflating: merged_data.csv         


In [ ]:
!pip install gluonts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.4 MB/s eta 0:00:00


In [ ]:
pip install mxnet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 45.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [ ]:
pip install mxnet --upgrade

In [ ]:
import numpy as np
np.bool = np.bool_

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import gluon
import json
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.mx import DeepAREstimator
from gluonts.mx.trainer import Trainer
import itertools

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [ ]:
'''
Set the random seed for both MXNet (mx) and NumPy (np) libraries to ensure reproducibility of results.
'''

mx.random.seed(7)
np.random.seed(7)

In [ ]:
'''
Read the CSV file named "merged_data.csv" into a DataFrame named df_merged using the pd.read_csv function from the pandas library.
Ensure that the 'settlement_date' column is interpreted as datetime by converting it using the pd.to_datetime function.
'''
df_merged = pd.read_csv("merged_data.csv")
df_merged['settlement_date'] = pd.to_datetime(df_merged['settlement_date'])

In [ ]:
'''
Set the index of the DataFrame df_merged to the 'settlement_date' column and store the result in a new DataFrame named df_model_3.

Define the split_from variable to determine the point of split between training and testing data. In this case, split_from is set to 90 days multiplied by 12, assuming hourly data.

Split the df_model_3 DataFrame into training and testing sets:
- train_data contains all data points from df_model_3 except for the last entries defined by split_from, representing the last 90 days' worth of data based on the assumption of hourly recordings.
- test_data consists of the last 90 days' worth of data from df_model_3, serving as the dataset for evaluating the model's performance on unseen data.
'''

df_model_3 = df_merged.set_index("settlement_date")

split_from = 90*12 #train test split is from 90days
train_data = df_model_3[:-split_from]
test_data = df_model_3[-split_from:]

In [ ]:
'''
Define the frequency of the time series data as "2H" (2 hours).
Specify the number of days to predict, and calculate the prediction_length by multiplying days_to_predict by 12.
Set the context_length to twice the prediction_length, assuming a context_length of 2 times the prediction_length. This provides a longer context window for the model to learn patterns and make predictions.
'''
freq = "2H"
days_to_predict=7
prediction_length = days_to_predict*12 #predicting 7 days
context_length = prediction_length *2

In [ ]:
'''
Define a parameter grid containing different combinations of hyperparameters for the DeepAR model, including epochs, num_batches_per_epoch, num_cells, num_layers, and dropout_rate.

Generate all possible combinations of parameters using itertools.product and store them in all_params.

Iterate over each parameter combination in all_params:
    - Define a DeepAR estimator with the current set of parameters.
    - Create training and testing datasets using ListDataset.
    - Train the model on the training dataset.
    - Generate forecasts on the testing dataset.
    - Evaluate the forecasts using the Evaluator and calculate the MAPE.
    - Append the MAPE to the list mapes.

Combine the parameters and corresponding MAPEs into a DataFrame named tuning_results.

Print the tuning_results DataFrame to display the results of hyperparameter tuning.
'''

param_grid = {
    'epochs': [8, 12],
    'num_batches_per_epoch': [100, 150],
    'num_cells': [40, 64],
    'num_layers': [2, 4],
    'dropout_rate':[0.1, 0.01]
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
mapes = []  # Store the MAPEs for each parameter combination here

# Use cross validation to evaluate all parameters
for params in all_params:
    # Define DeepAR estimator with given parameters
    estimator = DeepAREstimator(
        freq=freq,
        context_length=context_length,
        prediction_length=prediction_length,
        cardinality = [1,1],
        dropout_rate = params['dropout_rate'],
        use_feat_dynamic_real =True, # weather temp
        num_layers=params['num_layers'],
        num_cells=params['num_cells'],
        trainer=Trainer(
            epochs=params['epochs'],
            num_batches_per_epoch=params['num_batches_per_epoch']
        )
    )

    train_ds = ListDataset(
    [{"start":train_data.index[0],
      "target":train_data.tsd,
      "feat_dynamic_real": [train_data.temp],
      "feat_dynamic_cat": [train_data.is_holiday],
      }],
    freq=freq
)

    test_ds = ListDataset(
        [{"start":test_data.index[0],
          "target":test_data.tsd,
          "feat_dynamic_real": [test_data.temp],
          "feat_dynamic_cat": [test_data.is_holiday],
          }],
        freq=freq
    )

    # Train the model
    m = estimator.train(training_data=train_ds)
    # Generate forecasts
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,
        predictor=m,
        num_samples=100
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    # Get MAPE for the forecasts
    evaluator = Evaluator()
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts))
    mape = agg_metrics["MAPE"]
    mapes.append(mape)

# Combine parameters and corresponding MAPEs into a DataFrame
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes
print(tuning_results)


100%|██████████| 100/100 [00:36<00:00,  2.72it/s, epoch=8/8, avg_epoch_loss=8.29]
Running evaluation: 1it [00:00, 10.41it/s]
100%|██████████| 100/100 [00:36<00:00,  2.75it/s, epoch=8/8, avg_epoch_loss=8.41]
Running evaluation: 1it [00:00, 10.55it/s]
100%|██████████| 100/100 [01:01<00:00,  1.63it/s, epoch=8/8, avg_epoch_loss=8.38]
Running evaluation: 1it [00:00,  9.00it/s]
100%|██████████| 100/100 [01:03<00:00,  1.57it/s, epoch=8/8, avg_epoch_loss=8.27]
Running evaluation: 1it [00:00,  7.74it/s]
100%|██████████| 100/100 [00:43<00:00,  2.33it/s, epoch=8/8, avg_epoch_loss=8.21]
Running evaluation: 1it [00:00,  7.74it/s]
100%|██████████| 100/100 [00:42<00:00,  2.35it/s, epoch=8/8, avg_epoch_loss=8.28]
Running evaluation: 1it [00:00,  7.71it/s]
100%|██████████| 100/100 [01:12<00:00,  1.38it/s, epoch=8/8, avg_epoch_loss=8.32]
Running evaluation: 1it [00:00,  6.80it/s]
100%|██████████| 100/100 [01:12<00:00,  1.38it/s, epoch=8/8, avg_epoch_loss=8.4]
Running evaluation: 1it [00:00,  6.70it/s]
1

    epochs  num_batches_per_epoch  num_cells  num_layers  dropout_rate  \
0        8                    100         40           2          0.10   
1        8                    100         40           2          0.01   
2        8                    100         40           4          0.10   
3        8                    100         40           4          0.01   
4        8                    100         64           2          0.10   
5        8                    100         64           2          0.01   
6        8                    100         64           4          0.10   
7        8                    100         64           4          0.01   
8        8                    150         40           2          0.10   
9        8                    150         40           2          0.01   
10       8                    150         40           4          0.10   
11       8                    150         40           4          0.01   
12       8                    150     

In [ ]:
tuning_results

,epochs,num_batches_per_epoch,num_cells,num_layers,dropout_rate,mape
0,8,100,40,2,0.10,0.136152
1,8,100,40,2,0.01,0.102798
2,8,100,40,4,0.10,0.104653
3,8,100,40,4,0.01,0.097510
4,8,100,64,2,0.10,0.118332
5,8,100,64,2,0.01,0.139776
6,8,100,64,4,0.10,0.191124
7,8,100,64,4,0.01,0.148500
8,8,150,40,2,0.10,0.192642
9,8,150,40,2,0.01,0.154756
